## Global-Only Dashboard (Emotion + Relationship)

- 모든 에피소드를 `global_turn`으로 이어서 봅니다.
- 점선/라벨로 에피소드 경계를 표시합니다.
- 수민(또는 지정 캐릭터) 관점의 관계 변화를 함께 봅니다.

In [ ]:
from pathlib import Path
import re
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

DB_PATH = Path('data/simulation.db')
assert DB_PATH.exists(), f"DB not found: {DB_PATH}"
conn = sqlite3.connect(DB_PATH)


def q(sql, params=None):
    return pd.read_sql_query(sql, conn, params=params or {})


def _split_episode_trial(episode_id: str):
    m = re.match(r"^(.*?)(?:_trial(\d+))?$", str(episode_id))
    if not m:
        return str(episode_id), 0
    return m.group(1), int(m.group(2)) if m.group(2) else 0


def _episode_sort_key(episode_id: str):
    base, trial = _split_episode_trial(episode_id)
    m = re.search(r"ep(\d+)", base)
    ep_num = int(m.group(1)) if m else 9999
    return (ep_num, base, trial)


def _add_global_turn(df: pd.DataFrame, turn_col: str = 'turn', ts_col: str = 'timestamp') -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out['episode_base'] = out['episode_id'].map(lambda x: _split_episode_trial(x)[0])
    out['trial_num'] = out['episode_id'].map(lambda x: _split_episode_trial(x)[1])
    out['_ep_key'] = out['episode_id'].map(_episode_sort_key)
    out[turn_col] = pd.to_numeric(out[turn_col], errors='coerce').fillna(0).astype(int)
    sort_cols = ['_ep_key', turn_col] + ([ts_col] if ts_col in out.columns else [])
    out = out.sort_values(sort_cols, kind='stable').reset_index(drop=True)
    out['global_turn'] = out.index + 1
    return out.drop(columns=['_ep_key'])


def _episode_boundaries(df: pd.DataFrame, xcol: str = 'global_turn') -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame(columns=['episode_id', 'start', 'end'])
    return (
        df.groupby('episode_id', as_index=False)[xcol]
          .agg(start='min', end='max')
          .sort_values('start')
          .reset_index(drop=True)
    )


def _draw_episode_boundaries(ax, boundaries: pd.DataFrame):
    if boundaries.empty:
        return
    for _, r in boundaries.iloc[:-1].iterrows():
        ax.axvline(r['end'], color='gray', linestyle='--', linewidth=0.8, alpha=0.45)
    y_top = ax.get_ylim()[1]
    for _, r in boundaries.iterrows():
        mid = (r['start'] + r['end']) / 2.0
        ax.text(mid, y_top, str(r['episode_id']).replace('_trial', ' t'),
                ha='center', va='bottom', fontsize=8, alpha=0.7)


def load_emotions_all(agent_id: str | None = None, emotion_type: str | None = None, episode_base: str | None = None) -> pd.DataFrame:
    sql = """
    SELECT agent_id, interaction_id, episode_id, turn, emotion_type, intensity, timestamp
    FROM emotions
    WHERE 1=1
    """
    params = {}
    if agent_id:
        sql += " AND agent_id = :agent_id"
        params['agent_id'] = agent_id
    if emotion_type:
        sql += " AND emotion_type = :emotion_type"
        params['emotion_type'] = emotion_type
    df = q(sql, params)
    if df.empty:
        return df
    if episode_base:
        df = df[df['episode_id'].str.startswith(episode_base)]
    df['intensity'] = pd.to_numeric(df['intensity'], errors='coerce')
    return _add_global_turn(df, 'turn')


def load_relationships_all(episode_base: str | None = None) -> pd.DataFrame:
    sql = """
    SELECT agent1_id, agent2_id, value, episode_id, turn, updated_at AS timestamp
    FROM relationships
    """
    df = q(sql)
    if df.empty:
        return df
    if episode_base:
        df = df[df['episode_id'].str.startswith(episode_base)]
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    return _add_global_turn(df, 'turn')


def plot_emotion_global_labeled(emotion_type: str, agent_ids=None, episode_base: str | None = None, figsize=(13, 4)):
    df = load_emotions_all(emotion_type=emotion_type, episode_base=episode_base)
    if agent_ids:
        df = df[df['agent_id'].isin(agent_ids)]
    if df.empty:
        print('No emotion data for', emotion_type)
        return

    boundaries = _episode_boundaries(df)
    x_min = int(df['global_turn'].min())
    x_max = int(df['global_turn'].max())

    fig, ax = plt.subplots(figsize=figsize)
    for aid, g in df.groupby('agent_id'):
        s = g.groupby('global_turn', as_index=True)['intensity'].mean().sort_index()
        ax.plot(s.index, s.values, label=aid, linewidth=1.6, marker='o', markersize=3)

    _draw_episode_boundaries(ax, boundaries)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0.0, 1.0)
    ax.set_xlabel('global_turn (concatenated)')
    ax.set_ylabel('intensity')
    ax.set_title(f"Emotion='{emotion_type}' | {episode_base if episode_base else 'ALL episodes'}")
    ax.legend(loc='best', ncol=2, fontsize=8)
    ax.grid(True, linewidth=0.3)
    plt.tight_layout()
    plt.show()


def plot_all_emotions_global(agent_ids=None, episode_base: str | None = None, figsize=(13, 4)):
    em = load_emotions_all(episode_base=episode_base)
    if agent_ids:
        em = em[em['agent_id'].isin(agent_ids)]
    if em.empty:
        print('No emotion data.')
        return

    emotions = sorted(em['emotion_type'].dropna().unique().tolist())
    boundaries = _episode_boundaries(em)
    x_min = int(em['global_turn'].min())
    x_max = int(em['global_turn'].max())

    for emo in emotions:
        sub = em[em['emotion_type'] == emo]
        fig, ax = plt.subplots(figsize=figsize)
        for aid, g in sub.groupby('agent_id'):
            s = g.groupby('global_turn', as_index=True)['intensity'].mean().sort_index()
            ax.plot(s.index, s.values, label=aid, linewidth=1.4, marker='o', markersize=3)

        _draw_episode_boundaries(ax, boundaries)
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(0.0, 1.0)
        ax.set_title(f"Emotion='{emo}' | {episode_base if episode_base else 'ALL episodes'}")
        ax.set_xlabel('global_turn (concatenated)')
        ax.set_ylabel('intensity')
        ax.legend(loc='best', ncol=2, fontsize=8)
        ax.grid(True, linewidth=0.3)
        plt.tight_layout()
        plt.show()


def list_anchor_relationships(anchor_id: str = 'kim_sumin', episode_base: str | None = None) -> pd.DataFrame:
    rel = load_relationships_all(episode_base=episode_base)
    if rel.empty:
        return rel

    rows = []
    for _, r in rel.iterrows():
        if r['agent1_id'] == anchor_id:
            rows.append({'other_id': r['agent2_id'], 'value': r['value'], 'episode_id': r['episode_id'], 'global_turn': r['global_turn']})
        elif r['agent2_id'] == anchor_id:
            rows.append({'other_id': r['agent1_id'], 'value': r['value'], 'episode_id': r['episode_id'], 'global_turn': r['global_turn']})

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    return (
        out.groupby('other_id', as_index=False)
           .agg(updates=('value', 'count'), episodes=('episode_id', 'nunique'),
                min_value=('value', 'min'), max_value=('value', 'max'), last_value=('value', 'last'))
           .sort_values(['updates', 'episodes'], ascending=[False, False])
    )


def plot_anchor_relationships_global(anchor_id: str = 'kim_sumin', top_k: int = 6, episode_base: str | None = None, figsize=(13, 6)):
    rel = load_relationships_all(episode_base=episode_base)
    if rel.empty:
        print('No relationship data.')
        return

    rows = []
    for _, r in rel.iterrows():
        if r['agent1_id'] == anchor_id:
            rows.append({'other_id': r['agent2_id'], 'value': r['value'], 'global_turn': r['global_turn'], 'episode_id': r['episode_id']})
        elif r['agent2_id'] == anchor_id:
            rows.append({'other_id': r['agent1_id'], 'value': r['value'], 'global_turn': r['global_turn'], 'episode_id': r['episode_id']})

    df = pd.DataFrame(rows)
    if df.empty:
        print(f'No relationships involving anchor={anchor_id}')
        return

    top = (
        df.groupby('other_id', as_index=False)
          .agg(updates=('value', 'count'))
          .sort_values('updates', ascending=False)
          .head(top_k)
    )
    keep = set(top['other_id'])
    df = df[df['other_id'].isin(keep)]

    fig, ax = plt.subplots(figsize=figsize)
    for oid, g in df.groupby('other_id'):
        s = g.groupby('global_turn', as_index=True)['value'].mean().sort_index()
        ax.plot(s.index, s.values, marker='o', markersize=3, linewidth=1.5, label=oid)

    _draw_episode_boundaries(ax, _episode_boundaries(df))
    ax.set_xlabel('global_turn (concatenated)')
    ax.set_ylabel('relationship value')
    ax.set_title(f'Relationships from anchor={anchor_id} | {episode_base if episode_base else "ALL episodes"}')
    ax.legend(loc='best', ncol=2, fontsize=8)
    ax.grid(True, linewidth=0.3)
    plt.tight_layout()
    plt.show()


# Run examples
plot_all_emotions_global()
print(list_anchor_relationships('kim_sumin').head(10))
plot_anchor_relationships_global('kim_sumin', top_k=6)

